# General integration (quad)
The function quad is provided to integrate a function of one variable between two points.  For example, suppose you wish to integrate a bessel function jv(2.5, x) along the interval 

#https://docs.scipy.org/doc/scipy/reference/tutorial/integrate.html

In [2]:

import scipy.integrate as integrate
import scipy.special as special
result = integrate.quad(lambda x: special.jv(2.5,x), 0, 4.5)
result


(1.1178179380783244, 7.866317216380707e-09)

In [3]:
from numpy import sqrt, sin, cos, pi
I = sqrt(2/pi)*(18.0/27*sqrt(2)*cos(4.5) - 4.0/27*sqrt(2)*sin(4.5) +
                sqrt(2*pi) * special.fresnel(3/sqrt(pi))[0])
I

1.117817938088701

In [4]:
print(abs(result[0]-I))

1.0376588477356563e-11


The first argument to quad is a “callable” Python object (i.e. a function, method, or class instance). Notice the use of a lambda- function in this case as the argument. The next two arguments are the limits of integration. The return value is a tuple, with the first element holding the estimated value of the integral and the second element holding an upper bound on the error

In [6]:
from scipy.integrate import quad
def integrand(x, a, b):
     return a*x**2 + b
a = 2
b = 1
I = quad(integrand, 0, 1, args=(a,b))
I

(1.6666666666666667, 1.8503717077085944e-14)

In [10]:
from scipy.integrate import quad
import numpy as np
def integrand(t, n, x):
    return np.exp(-x*t) / t**n

In [11]:
def expint(n, x):
    return quad(integrand, 1, np.inf, args=(n, x))[0]


In [12]:
vec_expint = np.vectorize(expint)

In [13]:
vec_expint(3, np.arange(1.0, 4.0, 0.5))

import scipy.special as special
special.expn(3, np.arange(1.0,4.0,0.5))

array([0.10969197, 0.05673949, 0.03013338, 0.01629537, 0.00893065,
       0.00494538])

In [14]:
result = quad(lambda x: expint(3, x), 0, np.inf)
print(result)

(0.3333333333366853, 1.3274031145043084e-09)


In [15]:
I3 = 1.0/3.0
print(I3)


0.3333333333333333


In [16]:
print(I3 - result[0])

-3.3519853559482726e-12


# General multiple integration (dblquad, tplquad, nquad)¶

In [17]:
from scipy.integrate import quad, dblquad
def I(n):
    return dblquad(lambda t, x: np.exp(-x*t)/t**n, 0, np.inf, lambda x: 1, lambda x: np.inf)

In [18]:
print(I(4))

print(I(3))

print(I(2))


(0.2500000000043577, 1.29830334693681e-08)
(0.3333333333366853, 1.3888461883425516e-08)
(0.499999999909358, 1.4640839512484866e-08)


In [19]:
from scipy.integrate import dblquad
area = dblquad(lambda x, y: x*y, 0, 0.5, lambda x: 0, lambda x: 1-2*x)
area

(0.010416666666666668, 4.101620128472366e-16)

In [20]:
from scipy import integrate
N = 5
def f(t, x):
   return np.exp(-x*t) / t**N

integrate.nquad(f, [[1, np.inf],[0, np.inf]])


(0.2000000000189363, 1.3682975855986121e-08)

In [21]:
from scipy import integrate
def f(x, y):
    return x*y

def bounds_y():
    return [0, 0.5]

def bounds_x(y):
    return [0, 1-2*y]

integrate.nquad(f, [bounds_x, bounds_y])

(0.010416666666666668, 4.101620128472366e-16)

# Gaussian quadrature
A few functions are also provided in order to perform simple Gaussian quadrature over a fixed interval. The first is fixed_quad which performs fixed-order Gaussian quadrature. The second function is quadrature which performs Gaussian quadrature of multiple orders until the difference in the integral estimate is beneath some tolerance supplied by the user. These functions both use the module special.orthogonal which can calculate the roots and quadrature weights of a large variety of orthogonal polynomials (the polynomials themselves are available as special functions returning instances of the polynomial class — e.g. special.legendre).

Romberg Integration
Romberg’s method [WPR] is another method for numerically evaluating an integral. See the help function for romberg for further details.

Integrating using Samples
If the samples are equally-spaced and the number of samples available is  for some integer , then Romberg romb integration can be used to obtain high-precision estimates of the integral using the available samples. Romberg integration uses the trapezoid rule at step-sizes related by a power of two and then performs Richardson extrapolation on these estimates to approximate the integral with a higher-degree of accuracy.

In case of arbitrary spaced samples, the two functions trapz (defined in numpy [NPT]) and simps are available. They are using Newton-Coates formulas of order 1 and 2 respectively to perform integration. The trapezoidal rule approximates the function as a straight line between adjacent points, while Simpson’s rule approximates the function between three adjacent points as a parabola.

For an odd number of samples that are equally spaced Simpson’s rule is exact if the function is a polynomial of order 3 or less. If the samples are not equally spaced, then the result is exact only if the function is a polynomial of order 2 or less.

In [22]:
import numpy as np
def f1(x):
   return x**2

def f2(x):
   return x**3

x = np.array([1,3,4])
y1 = f1(x)
from scipy.integrate import simps
I1 = simps(y1, x)
print(I1)

21.0


In [23]:
y2 = f2(x)
I2 = integrate.simps(y2, x)
print(I2)


61.5


# Ordinary differential equations (odeint)¶

In [27]:
from scipy.integrate import odeint
from scipy.special import gamma, airy
y1_0 = 1.0 / 3**(2.0/3.0) / gamma(2.0/3.0)
y0_0 = -1.0 / 3**(1.0/3.0) / gamma(1.0/3.0)
y0 = [y0_0, y1_0]
def func(y, t):
    return [t*y[1],y[0]]


In [28]:
def gradient(y, t):
    return [[0,t], [1,0]]


In [29]:
x = np.arange(0, 4.0, 0.01)
t = x
ychk = airy(x)[0]
y = odeint(func, y0, t)
y2 = odeint(func, y0, t, Dfun=gradient)

In [30]:
ychk[:36:6]

array([0.35502805, 0.33951139, 0.32406751, 0.30876307, 0.29365818,
       0.27880648])

In [31]:
y[:36:6,1]

array([0.35502805, 0.33951138, 0.32406749, 0.30876306, 0.29365817,
       0.27880648])

In [32]:
y2[:36:6,1]

array([0.35502805, 0.33951138, 0.32406749, 0.30876306, 0.29365817,
       0.27880648])